In [1]:
import xyzservices.providers as xyz
import pandas as pd
import numpy as np

from bokeh.models.annotations import Title

from bokeh.plotting import figure, show
from bokeh.tile_providers import get_provider,CARTODBPOSITRON
from bokeh.models import ColumnDataSource,ColorBar,LinearColorMapper,HoverTool,Triangle

from bokeh.palettes import Spectral6,Viridis10
from bokeh.transform import linear_cmap

from bokeh.layouts import row
from bokeh.io import output_file,save

In [2]:
df=pd.read_csv("Jul-4th-2022-06-37PM-Flight-Airdata.csv")
df.head(100)

,time(millisecond),datetime(utc),latitude,longitude,height_above_takeoff(feet),height_above_ground_at_drone_location(feet),ground_elevation_at_drone_location(feet),altitude_above_seaLevel(feet),height_sonar(feet),speed(mph),...,voltageCell4,voltageCell5,voltageCell6,current(A),battery_temperature(f),altitude(feet),ascent(feet),flycStateRaw,flycState,message
0,0,2022-07-04 22:37:18,40.441219,-79.946885,0.000000,Available with any HD 360 subscription,Available with any HD 360 subscription,927.607648,0.328084,0.0,...,0,0,0,0.000,76.82,927.607648,0.000000,41,Motors_Started,NaN
1,100,2022-07-04 22:37:18,40.441219,-79.946885,0.000000,NaN,NaN,927.607648,0.328084,0.0,...,0,0,0,0.000,76.82,927.607648,0.000000,41,Motors_Started,NaN
2,200,2022-07-04 22:37:18,40.441219,-79.946885,0.000000,NaN,NaN,927.607648,0.328084,0.0,...,0,0,0,0.000,76.82,927.607648,0.000000,41,Motors_Started,NaN
3,300,2022-07-04 22:37:18,40.441219,-79.946885,0.000000,NaN,NaN,927.607648,0.328084,0.0,...,0,0,0,0.000,76.82,927.607648,0.000000,41,Motors_Started,NaN
4,400,2022-07-04 22:37:18,40.441219,-79.946885,0.000000,NaN,NaN,927.607648,0.328084,0.0,...,0,0,0,0.000,76.82,927.607648,0.000000,41,Motors_Started,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,9500,2022-07-04 22:37:27,40.441219,-79.946884,22.637796,NaN,NaN,950.245444,24.278216,0.0,...,0,0,0,6.258,77.00,950.245444,22.637796,6,P-GPS,NaN
96,9600,2022-07-04 22:37:27,40.441219,-79.946884,22.965880,NaN,NaN,950.573528,24.606300,0.0,...,0,0,0,6.258,77.00,950.573528,22.965880,6,P-GPS,NaN
97,9700,2022-07-04 22:37:27,40.441219,-79.946884,23.622048,NaN,NaN,951.229696,25.262468,0.0,...,0,0,0,6.258,77.00,951.229696,23.622048,6,P-GPS,NaN
98,9800,2022-07-04 22:37:27,40.441219,-79.946884,24.278216,NaN,NaN,951.885864,25.918636,0.0,...,0,0,0,6.821,77.18,951.885864,24.278216,6,P-GPS,NaN


In [3]:
def wgs84_to_web_mercator(df, lon="LON", lat="LAT"):
    k = 6378137
    df["LON"] = df[lon] * (k * np.pi/180.0)
    df["LAT"] = np.log(np.tan((90 + df[lat]) * np.pi/360.0)) * k

    return df

In [4]:
df[df['flycStateRaw'].diff()!=0]

,time(millisecond),datetime(utc),latitude,longitude,height_above_takeoff(feet),height_above_ground_at_drone_location(feet),ground_elevation_at_drone_location(feet),altitude_above_seaLevel(feet),height_sonar(feet),speed(mph),...,voltageCell4,voltageCell5,voltageCell6,current(A),battery_temperature(f),altitude(feet),ascent(feet),flycStateRaw,flycState,message
0,0,2022-07-04 22:37:18,40.441219,-79.946885,0.000000,Available with any HD 360 subscription,Available with any HD 360 subscription,927.607648,0.328084,0.000000,...,0,0,0,0.000,76.82,927.607648,0.000000,41,Motors_Started,NaN
15,1500,2022-07-04 22:37:19,40.441217,-79.946884,0.000000,NaN,NaN,927.607648,0.328084,0.000000,...,0,0,0,0.000,76.82,927.607648,0.000000,11,AutoTakeoff,NaN
44,4400,2022-07-04 22:37:22,40.441216,-79.946883,3.280840,NaN,NaN,930.888488,3.608924,0.000000,...,0,0,0,4.646,76.82,930.888488,3.280840,6,P-GPS,NaN
581,58100,2022-07-04 22:38:16,40.439814,-79.947267,133.858272,NaN,NaN,1061.465920,83.661420,0.000000,...,0,0,0,6.921,83.12,1061.465920,133.858272,37,Quick_Shot,NaN
1136,113600,2022-07-04 22:39:11,40.438982,-79.946921,134.186356,NaN,NaN,1061.794004,83.661420,0.715821,...,0,0,0,7.174,87.44,1061.794004,134.186356,17,Joystick,NaN
1137,113700,2022-07-04 22:39:11,40.438982,-79.946921,134.186356,NaN,NaN,1061.794004,83.661420,0.492127,...,0,0,0,7.174,87.44,1061.794004,134.186356,6,P-GPS,NaN
3450,345000,2022-07-04 22:43:03,40.441212,-79.946895,-12.795276,NaN,NaN,914.812372,2.296588,0.000000,...,0,0,0,7.501,102.02,914.812372,-12.795276,12,AutoLanding,NaN


In [5]:
df1=wgs84_to_web_mercator(df,'longitude','latitude')

In [6]:
df1["LAT"].values.shape
np.array(['red' for i in range(len(df1))]).shape

(3500,)

In [7]:
# df[(df['message'].isna()==False)][['flycState','message']]

# df['message_filt']=df.apply(lambda x:1 if isinstance( x['message'], str) else 0,axis=1)
df['PrintMessage']=df.apply(lambda x:x['message'] if isinstance( x['message'],str) else x['flycState'],axis=1)


# df[(df['flycStateRaw'].diff()!=0)|(df['message'].isna()==False)][['flycState','message','message_filt','PrintMessage']]

In [8]:
df['message_filt']=df.apply(lambda x:1 if isinstance( x['message'], str) else 0,axis=1)
np.array(df[df['message_filt']==1].index)

array([   7,   29, 3486], dtype=int64)

In [9]:
tile_provider = get_provider(xyz.OpenStreetMap.Mapnik)

source = ColumnDataSource(
    data=dict(lat=df1["LAT"].values,
              lon=df1["LON"].values)
)
source_message = ColumnDataSource(
    data=dict(lat=df1["LAT"][(df['flycStateRaw'].diff()!=0)|(df['message'].isna()==False)].values,
              lon=df1["LON"][(df['flycStateRaw'].diff()!=0)|(df['message'].isna()==False)].values,
              message=df[(df['flycStateRaw'].diff()!=0)|(df['message'].isna()==False)]['PrintMessage'].values)
)
# range bounds supplied in web mercator coordinates
p = figure(x_axis_type="mercator", y_axis_type="mercator")
p.add_tile(tile_provider)

p.circle(x="lon", y="lat", size=2, line_color="yellow",color="yellow", fill_color="yellow", fill_alpha=0.8, source=source)

r=p.triangle(x="lon", y="lat", size=8, color="red", fill_alpha=1, source=source_message,legend_label='Drone State')
Labels=[('Message','@message')]
routeMapHover = HoverTool(tooltips=Labels,mode='mouse',point_policy='follow_mouse',renderers=[r])

p.tools.append(routeMapHover)

p.xaxis.axis_label = 'Longitude'
p.yaxis.axis_label = 'Latitude'

t = Title()
t.text = 'Flight path - overlay on detailed map'
p.title = t


# show(p)

In [11]:
tile_provider = get_provider(CARTODBPOSITRON)

source_battery = ColumnDataSource(
    data=dict(lat=df1["LAT"].values,
              lon=df1["LON"].values,
            battery_percent=df['battery_percent'])
)

source_video = ColumnDataSource(
    data=dict(lat=df1["LAT"][df['isVideo']==1].values,
              lon=df1["LON"][df['isVideo']==1].values,)
)

source_message = ColumnDataSource(
    data=dict(lat=df1["LAT"][(df['flycStateRaw'].diff()!=0)|(df['message'].isna()==False)].values,
              lon=df1["LON"][(df['flycStateRaw'].diff()!=0)|(df['message'].isna()==False)].values,
              message=df[(df['flycStateRaw'].diff()!=0)|(df['message'].isna()==False)]['PrintMessage'].values)
)

p_battery = figure(x_axis_type="mercator", y_axis_type="mercator")
p_battery.add_tile(tile_provider)

mapper_battery = linear_cmap(field_name='battery_percent', palette=Viridis10 ,low=df['battery_percent'].min() ,high=df['battery_percent'].max())
color_battery = LinearColorMapper(palette = "Viridis10",low = df['battery_percent'].min(), high = df['battery_percent'].max())

p_battery.circle(x="lon", y="lat", size=2, line_color="blue",color="blue", fill_alpha=1, source=source_video,legend_label='Video on')
p_battery.circle(x="lon", y="lat", size=2, line_color=mapper_battery,color=mapper_battery, fill_alpha=1, source=source_battery,legend_label='Battery-trail %')
r=p_battery.triangle(x="lon", y="lat", size=8, color="red", fill_alpha=1, source=source_message,legend_label='Drone State')
Labels=[('Message','@message')]
routeMapHover = HoverTool(tooltips=Labels,mode='mouse',point_policy='follow_mouse',renderers=[r])
    #associate the tooltip and map
p_battery.tools.append(routeMapHover)

cb = ColorBar(color_mapper = color_battery, location = (5,6))
 
# Adding the color bar to the right side
p_battery.add_layout(cb, 'right')
p_battery.legend.click_policy = "hide"
p_battery.legend.location = 'top_left'


p_battery.xaxis.axis_label = 'Longitude'
p_battery.yaxis.axis_label = 'Latitude'

t = Title()
t.text = 'Battery percentage(color-bar)|video state over jouney'
p_battery.title = t


In [12]:
tile_provider = get_provider(CARTODBPOSITRON)

source_battery = ColumnDataSource(
    data=dict(lat=df1["LAT"].values,
              lon=df1["LON"].values,
            battery_percent=df['altitude_above_seaLevel(feet)'])
)

source_video = ColumnDataSource(
    data=dict(lat=df1["LAT"][df['isVideo']==1].values,
              lon=df1["LON"][df['isVideo']==1].values,)
)

source_message = ColumnDataSource(
    data=dict(lat=df1["LAT"][(df['flycStateRaw'].diff()!=0)|(df['message'].isna()==False)].values,
              lon=df1["LON"][(df['flycStateRaw'].diff()!=0)|(df['message'].isna()==False)].values,
              message=df[(df['flycStateRaw'].diff()!=0)|(df['message'].isna()==False)]['PrintMessage'].values)
)
# range bounds supplied in web mercator coordinates
p_altitude = figure(x_axis_type="mercator", y_axis_type="mercator")
p_altitude.add_tile(tile_provider)

mapper_battery = linear_cmap(field_name='battery_percent', palette=Viridis10 ,low=df['altitude_above_seaLevel(feet)'].min() ,high=df['altitude_above_seaLevel(feet)'].max())
color_battery = LinearColorMapper(palette = "Viridis10",low = df['altitude_above_seaLevel(feet)'].min(), high = df['altitude_above_seaLevel(feet)'].max())

p_altitude.circle(x="lon", y="lat", size=2, line_color="blue",color="blue", fill_alpha=1, source=source_video,legend_label='Video on')
p_altitude.circle(x="lon", y="lat", size=2, line_color=mapper_battery,color=mapper_battery, fill_alpha=1, source=source_battery,legend_label='Altitude-ft')
r=p_altitude.triangle(x="lon", y="lat", size=8, color="red", fill_alpha=1, source=source_message,legend_label='Drone State')
Labels=[('Message','@message')]
routeMapHover = HoverTool(tooltips=Labels,mode='mouse',point_policy='follow_mouse',renderers=[r])
    #associate the tooltip and map
p_altitude.tools.append(routeMapHover)

cb = ColorBar(color_mapper = color_battery, location = (5,6))
 
# Adding the color bar to the right side
p_altitude.add_layout(cb, 'right')
p_altitude.legend.click_policy = "hide"
p_altitude.legend.location = 'top_left'


p_altitude.xaxis.axis_label = 'Longitude'
p_altitude.yaxis.axis_label = 'Latitude'

t = Title()
t.text = 'Altitude above sea level - ft(color-bar)|video state over jouney'
p_altitude.title = t

In [13]:
mapCollection=row(p, p_battery,p_altitude)
output_file("drone_map.html")
save(mapCollection)

'C:\\Users\\krish\\Downloads\\bb\\Drone\\HW1\\DJI Flight Records\\op\\drone_map.html'